In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('homework_5') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/01 04:21:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
df_yellow = spark.read.option("header","true").parquet('/home/fabianphng/data-engineering-zoomcamp/05-batch/data/hmwk/yellow_tripdata_2024-10.parquet')

In [8]:
df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [6]:
df_yellow.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [7]:
df_yellow.repartition(4).write.parquet('/home/fabianphng/data-engineering-zoomcamp/05-batch/data/hmwk/repartitioned')

In [10]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [8]:
df_yellow.registerTempTable('yellow')

/home/fabianphng/spark/spark-3.4.4-bin-hadoop3/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [23]:
df_result_count = spark.sql("""
Select count(*) from yellow
where date(yellow.tpep_pickup_datetime) = '2024-10-15' and date(yellow.tpep_dropoff_datetime) = '2024-10-15';
""")

In [24]:
df_result_count.show()

+--------+
|count(1)|
+--------+
|  127993|
+--------+



In [21]:
df_result_longest = spark.sql("""
SELECT MAX(TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime)) / 3600 AS max_duration_in_hours
FROM yellow;
""")

In [22]:
df_result_longest.show()

+---------------------+
|max_duration_in_hours|
+---------------------+
|   162.61777777777777|
+---------------------+



In [25]:
df_zonelookup = spark.read.option("header","true").csv("/home/fabianphng/data-engineering-zoomcamp/05-batch/data/hmwk/taxi_zone_lookup.csv")

In [26]:
df_zonelookup.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [30]:
df_zonelookup.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [27]:
df_zonelookup.createOrReplaceTempView("zones")

In [42]:
df_result_leastPU = spark.sql("""
select count(yellow.PULocationID), zones.zone from yellow
inner join zones on zones.LocationID = yellow.PULocationID
group by zones.zone order by count(PULocationID);
""")

In [43]:
df_result_leastPU.show()

+-------------------+--------------------+
|count(PULocationID)|                zone|
+-------------------+--------------------+
|                  1|Governor's Island...|
|                  2|       Rikers Island|
|                  2|       Arden Heights|
|                  3|         Jamaica Bay|
|                  3| Green-Wood Cemetery|
|                  4|Charleston/Totten...|
|                  4|   Rossville/Woodrow|
|                  4|       West Brighton|
|                  4|       Port Richmond|
|                  4|Eltingville/Annad...|
|                  6|         Great Kills|
|                  6|        Crotona Park|
|                  7|     Mariners Harbor|
|                  7|Heartland Village...|
|                  9|Saint George/New ...|
|                  9|             Oakwood|
|                 10|New Dorp/Midland ...|
|                 10|       Broad Channel|
|                 12|         Westerleigh|
|                 12|     Pelham Bay Park|
+----------